In [1]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

True

# Firebase Init

In [2]:
service_account_info = {
    "type": os.getenv("FIREBASE_TYPE"),
    "project_id": os.getenv("FIREBASE_PROJECT_ID"),
    "private_key_id": os.getenv("FIREBASE_PRIVATE_KEY_ID"),
    "private_key": os.getenv("FIREBASE_PRIVATE_KEY"),
    "client_email": os.getenv("FIREBASE_CLIENT_EMAIL"),
    "client_id": os.getenv("FIREBASE_CLIENT_ID"),
    "auth_uri": os.getenv("FIREBASE_AUTH_URI"),
    "token_uri": os.getenv("FIREBASE_TOKEN_URI"),
    "auth_provider_x509_cert_url": os.getenv("FIREBASE_AUTH_PROVIDER_X509_CERT_URL"),
    "client_x509_cert_url": os.getenv("FIREBASE_CLIENT_X509_CERT_URL")
}

In [3]:
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred, {
    'storageBucket': 'soccer-shop-app.firebasestorage.app',
    'databaseURL': 'https://soccer-shop-app-default-rtdb.firebaseio.com/',
})

In [4]:
bucket = storage.bucket()

# Upload Data

In [5]:
image_folder_path = './products/images/'

In [6]:
products_collection = db.reference('products')

In [14]:
df = pd.read_json('products/products.jsonl', lines=True)
df.head(2)

,name,category,description,price,rating,image_path
0,Barcelona Crew Socks,Crew Socks,"Comfortable and stylish, these Barcelona Crew ...",12.99,4.5,barcelona_crew_socks.jpg
1,Barcelona Women's Classic Cleats,Women's Cleats,"Designed for performance and style, the Barcel...",79.99,4.7,barcelona_womens_classic_cleats.jpg


In [15]:
def upload_image(bucket, image_path):
    image_name = image_path.split('/')[-1]
    blob = bucket.blob(f'product_images/{image_name}')
    # upload image
    blob.upload_from_filename(image_path)

    # Make the image publicly accessible and get its URL
    blob.make_public()
    return blob.public_url

In [16]:
df.head(2)

,name,category,description,price,rating,image_path
0,Barcelona Crew Socks,Crew Socks,"Comfortable and stylish, these Barcelona Crew ...",12.99,4.5,barcelona_crew_socks.jpg
1,Barcelona Women's Classic Cleats,Women's Cleats,"Designed for performance and style, the Barcel...",79.99,4.7,barcelona_womens_classic_cleats.jpg


In [21]:
for index, row in df.iterrows():
    print(index,row['name'])

    image_path = os.path.join(image_folder_path, row['image_path'])
    image_url = upload_image(bucket, image_path)
    product_data = row.to_dict()
    product_data.pop('image_path')
    product_data['image_url'] = image_url

    # Add to Firestore
    products_collection.push().set(product_data)

0 Barcelona Crew Socks
1 Barcelona Women's Classic Cleats
2 Barcelona Men's Pro Cleats
3 Barcelona Women's Precision Cleats
4 Barcelona Men's Eco Cleats
5 Barcelona Men's Speed Cleats
6 Barcelona Women's Advanced Cleats
7 Barcelona Women's Speed Cleats
8 Barcelona Elite Socks
9 Barcelona Keeper Gloves
10 Barcelona Eco Keeper Gloves
11 Barcelona Performance Socks
12 Barcelona Dynamic Socks
13 Barcelona Team Wristband
14 Barcelona Team Scarf
